In [199]:
# Model design
import agentpy as ap

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import IPython

In [200]:
class dirtyRoomModel(ap.Model):

    def setup(self):

        # Create agents (Rooms)
        n_rooms = int(self.p['Dirty Rooms Density'] * (self.p.N **2))
        n_rooms = self.agents = ap.AgentList(self, n_rooms)

        # Create grid (Rooms)
        self.dirtyRoom = ap.Grid(self, [self.p.N, self.p.M ], track_empty=True)
        self.dirtyRoom.add_agents(n_rooms, random=True, empty=True)


        # Initiate a dynamic variable for all rooms
        # Condition 0: Dirty, 1: Cleaning, 2: Cleaned
        self.agents.condition = 0
        # Start the cleaning from the left side of the grid
        starting_rooms = self.dirtyRoom.agents[1, 1]
        starting_rooms.condition = 1

    def step(self):

        # Select burning n_rooms
        dirtyRoom = self.agents.select(self.agents.condition == 1)
        
        # Spread fire
        for room in dirtyRoom:
            for neighbor in self.dirtyRoom.neighbors(room):
                if neighbor.condition == 0:
                    neighbor.condition = 1 # Neighbor starts burning
            room.condition = 2 # room burns out

        # Stop simulation if no fire is left
        if len(dirtyRoom) == 0:
            self.stop()

    def end(self):

        # Document a measure at the end of the simulation
        dirtyRoom = len(self.agents.select(self.agents.condition == 2))
        self.report('Percentage of dirty rooms',
                    dirtyRoom / len(self.agents))

In [201]:
# Define parameters

parameters = {
    'Dirty Rooms Density': 1.5, # Percentage of grid covered by dirt
    'N': 30, # Height and length of the grid
    'M': 80,
    'steps': 300,
}

In [202]:
# Create single-run animation with custom colors

def animation_plot(model, ax):
    attr_grid = model.dirtyRoom.attr_grid('condition')
    color_dict = {0:'#7FC97F', 1:'#d62c2c', 2:'#e5e5e5', None:'#d5e5d5'}
    ap.gridplot(attr_grid, ax=ax, color_dict=color_dict, convert=True)
    ax.tick_params(axis='x', colors='black')    
    ax.tick_params(axis='y', colors='black')
    ax.set_title(f"Simulation of a cleaning robot\n"
                 f"Time-step: {model.t}, Rooms left: "
                 f"{len(model.agents.select(model.agents.condition == 0))}", color= 'black')

fig, ax = plt.subplots()
model = dirtyRoomModel(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml(fps=15))